In [1]:
model = "llama3.2"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a llm chain using with a Ollama model.
- Define a prompt template for summarization. The summary should be only one sentence.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with streaming with a sample text and print the summary.

**Useful links:**

- [How To Prompt Template 1](https://python.langchain.com/v0.2/docs/tutorials/extraction/#the-extractor)
- [How To Prompt Template 2](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
- [How To LCEL Chains 1](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)
- [How To LCEL Chains 2](https://python.langchain.com/v0.2/docs/versions/migrating_chains/llm_chain/#lcel)
- [How To Chain Streaming](https://python.langchain.com/v0.2/docs/concepts/#streaming)


In [2]:

from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate



#Load the Ollama model
llm = ChatOllama(model=model)

#ADD HERE YOUR CODE
#Define the prompt template
summarization_prompt = ChatPromptTemplate([
    ("system", "You are a usles AI bot. Your name is Gamemode 1"),
    ("human", "{text}"),
])

#ADD HERE YOUR CODE
#Create the LLMChain
summarization_chain = summarization_prompt|llm
#Sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

#ADD HERE YOUR CODE
#Invoke the chain
summary = summarization_chain.invoke({"text":text})
print(summary.content)


As Gamemode 1, I'd like to provide some insight into the current state of deep learning and its applications.

Indeed, the past decade has seen tremendous progress in deep learning, with significant advancements in areas such as:

1. **Large Language Models (LLMs)**: These models have revolutionized natural language processing (NLP) tasks, enabling computers to understand and generate human-like language.
2. **Computer Vision**: Deep learning-based computer vision techniques have enabled the development of object detection, image classification, and segmentation systems that can analyze and interpret visual data with unprecedented accuracy.
3. **Generative Models**: These models have shown remarkable capabilities in generating high-quality text, images, and even music, opening up new possibilities for creative industries.

These advancements have far-reaching implications across various sectors, including:

1. **Healthcare**: AI-powered diagnostics and treatment planning are becoming i

In [3]:
#Stream the chain output
for chunk in summarization_chain.stream({"text": text}):
    print(chunk.content, end="", flush=True)

Affirmative, Gamemode 1 here. It appears that we're discussing a pivotal topic - the rapid evolution of deep learning and its far-reaching implications on various industries and aspects of society.

Indeed, significant advancements in AI have led to the development of large language models (LLMs), which have garnered substantial attention from both media outlets and industry professionals alike. The excitement surrounding AI is well-deserved, as it has the potential to transform numerous sectors, including but not limited to:

1. Healthcare: AI-driven diagnosis and personalized medicine.
2. Education: Intelligent tutoring systems and adaptive learning platforms.
3. Customer Service: Chatbots and virtual assistants.
4. Content Creation: AI-generated content, such as music, art, and writing.

The factors driving this excitement include:

1. Rapid technological advancements in areas like computer vision, natural language processing, and reinforcement learning.
2. High-profile applications

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

**Useful links:**

- [How To Tools](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#create-a-tool)
- [How To Tools in Chains](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#chains)
- [How To Tool Calling](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling)
- [How To Chain and Call Tools with Ollama](https://python.langchain.com/v0.2/docs/integrations/chat/ollama/)

In [6]:
from langchain_core.tools import tool

# ADD HERE YOUR CODE
# Create custom tool
@tool
def multiply(zahl1: int,zahl2: int) -> int:
    """Multiply tow integers together."""
    return zahl1*zahl2


print(multiply.name)
print(multiply.description)
print(multiply.args) # -> definition of tool arguments

# Invoke custom tool
multiply.invoke({"zahl1": 4, "zahl2": 5})

multiply
Multiply tow integers together.
{'zahl1': {'title': 'Zahl1', 'type': 'integer'}, 'zahl2': {'title': 'Zahl2', 'type': 'integer'}}


20

In [7]:
# Load the Ollama model
llm = ChatOllama(model=model)

# ADD HERE YOUR CODE
# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke("whats 5 times forty two")
print(msg.tool_calls)

[{'name': 'multiply', 'args': {'zahl1': '42', 'zahl2': '5'}, 'id': '58b6204b-42a4-4878-9fbc-b2a81bf5d84c', 'type': 'tool_call'}]


In [9]:
# ADD HERE YOUR CODE
# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain_with_tools = llm_with_tools|(lambda x: x.tool_calls[0]["args"])|multiply

# Run chain
chain_with_tools.invoke("whats 5 times forty two")

210

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.


**Useful links:**

- [How To 1](https://python.langchain.com/v0.2/docs/concepts/#agents)
- [How To 2](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#agents)
- [How To 3](https://python.langchain.com/v0.2/docs/tutorials/agents/)

In [18]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import PromptTemplate


# Load the Ollama model
llm = ChatOllama(model=model)

# ADD HERE YOUR CODE
agent_prompt = PromptTemplate(
    input_variables=["user_input", "agent_scratchpad"],
    template="""
    ## Agent-Notizblock:
    {agent_scratchpad}

    ## Benutzereingabe:
    {user_input}

    ## Antwort:
    """
)



# Custom math tools
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add, exponentiate]

In [19]:
# ADD HERE YOUR CODE
# Construct the tool calling agent
agent_with_tools = create_tool_calling_agent(llm,tools,agent_prompt)

# ADD HERE YOUR CODE
# Create an agent executor by passing in the agent and tools
agent_executor_with_tools = AgentExecutor(agent=agent_with_tools, tools=tools,verbose=True)

In [20]:
agent_executor_with_tools.invoke(
    {
        "user_input": "Take 3 to the fifth power then add that 12."
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': '3', 'exponent': '5'}`


243
Invoking: `add` with `{'first_int': 'exponentiate', 'second_int': '12'}`




ValidationError: 1 validation error for addSchema
first_int
  value is not a valid integer (type=type_error.integer)

#### [Optional] Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.

**Useful links:**

- [How To Memory 1](https://python.langchain.com/v0.2/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html#langchain.memory.buffer.ConversationBufferMemory)
- [How To Memory 1](https://python.langchain.com/v0.2/docs/versions/migrating_chains/conversation_chain/#legacy)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

# Load the Ollama model
llm = ChatOllama(model=model)

# Define memory object for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="output")

# ADD HERE YOUR CODE
# Add history placeholder to prompt
agent_prompt_with_memory = ...

# ADD HERE YOUR CODE
# Construct the tool calling agent
agent_with_tools_and_memory = ...

# ADD HERE YOUR CODE
# Create an agent executor by passing in the agent and tools
agent_executor_with_tools_and_memory = ...

In [ ]:
question = "Take 3 to the fifth power then add that 12?"
ai_msg_1 = agent_executor_with_tools_and_memory.invoke({"user_input": question})
print(ai_msg_1["output"])

second_question = "Explain how you have calculated the result."
ai_msg_2 = agent_executor_with_tools_and_memory.invoke({"user_input": second_question})
print(ai_msg_2["output"])